In [1]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
import paramiko
import io
import datetime
import logging
pd.set_option('display.max_columns',30)

c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
log_file = f'Logs/MF_{datetime.datetime.now().strftime('%Y-%m-%d')}.log'

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler(log_file, mode='a'),
        logging.StreamHandler()
    ]
)

# A. Load Data

In [4]:
logging.info('Processing Start')

2025-01-17 14:35:52 - INFO - Processing Start


In [5]:
# setup date for querying
# last month
dataDt = datetime.date.today()-datetime.timedelta(days=1)
firstDate = dataDt.replace(day=1)
lastMonth = firstDate - datetime.timedelta(days=1)
lastMonth = lastMonth.strftime("%Y%m")

#last year
firstMonth = dataDt.replace(month=1, day=1)
lastYear = firstMonth - datetime.timedelta(days=1)
lastYear = lastYear.strftime("%Y%m")

# curr monthj
currMonth = firstDate.strftime("%Y%m")

## 1. DBA: Master Funding & MTD Aggregate

### a. Master Funding

In [6]:
# credentials
usernameDba = os.environ['UID']
passwordDba = os.environ['DBAPW']
dsnDba = '10.220.50.121:1661/ANALYTIC'

In [7]:
try:
    # create connection
    connection = oracledb.connect(user=usernameDba, password=passwordDba, dsn=dsnDba)

    # create engine
    engine = create_engine('oracle+oracledb://', creator=lambda: connection)

    # open and read the file as a single buffer
    mfQuery = open(r'MF.sql', 'r')
    sqlFile = mfQuery.read()
    # sqlFile.format(LM=lastMonth,LY=lastYear,CM=currMonth)
    mfQuery.close()

    # run query and store data in df (avgtime 2 min)
    logging.info('Running SQL Query')
    MF = pd.read_sql(sqlFile, engine)
    logging.info('SQL Query Finished')
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

2025-01-17 14:35:53 - INFO - Running SQL Query
2025-01-17 14:54:49 - INFO - SQL Query Finished


In [8]:
# change column names into UPPER CASE
MF.columns = [x.upper() for x in MF.columns]

In [9]:
# create source column for MASTER FUNDING data (TBL_BAL)
MF['SOURCE'] = 'TBL_BAL'

In [10]:
# convert BASE_DT to int for compatibility reasons
MF['BASE_DT'] = MF['BASE_DT'].astype(int)

# convert STATUS, SYSTEM_TYPE, and CTRL3 to string for compatibility
MF['STATUS'] = MF['STATUS'].astype(str)
MF['SYSTEM_TYPE'] = MF['SYSTEM_TYPE'].astype(str)
MF['CTRL3'] = MF['CTRL3'].astype(str)

In [11]:
MF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118043 entries, 0 to 118042
Data columns (total 45 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   BASE_DT             118043 non-null  int32         
 1   BASE_DT_PARSED      118043 non-null  datetime64[ns]
 2   BASE_YM             118043 non-null  object        
 3   AGREE_ID            118043 non-null  object        
 4   FLAG                118043 non-null  object        
 5   ACCT_NO             118043 non-null  object        
 6   ACCT_BR             118043 non-null  object        
 7   GCIF_CREATE_BR      24204 non-null   object        
 8   REGION              118041 non-null  object        
 9   AREA                118041 non-null  object        
 10  BRANCH              118041 non-null  object        
 11  GCIF_NO             118043 non-null  object        
 12  CIF_NO              118043 non-null  object        
 13  CUST_TYPE           24204 non

In [12]:
MF[['BASE_AMT_FIX','MTD','YTD']].describe()

,BASE_AMT_FIX,MTD,YTD
count,1.180430e+05,1.180430e+05,1.180430e+05
mean,6.541778e+08,2.997730e+06,2.997730e+06
std,8.496388e+09,3.883656e+09,3.883656e+09
min,0.000000e+00,-3.410001e+11,-3.410001e+11
25%,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.809258e+07,0.000000e+00,0.000000e+00
max,1.015641e+12,5.016699e+11,5.016699e+11


In [13]:
MF.columns

Index(['BASE_DT', 'BASE_DT_PARSED', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO',
       'ACCT_BR', 'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO',
       'CIF_NO', 'CUST_TYPE', 'PROD_NM', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME',
       'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD', 'RATE_DPK', 'BASE_AMT_FIX',
       'MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 'NPK_SALES', 'COLT',
       'BASE_AMT_ACCUM_MTD', 'INT_EXP_ACCUM_MTD', 'COF_MTD', 'DTD', 'MTD',
       'YTD', 'HIGH_COF_FLAG', 'SEGMENT_FIX', 'LOB_SORT', 'CASA_TD', 'DTD_10B',
       'MTD_10B', 'DIVISION', 'STATUS', 'SYSTEM_TYPE', 'CTRL3', 'BLOCK',
       'SOURCE'],
      dtype='object')

### b. Preprocess Bank Entities

In [14]:
# get working directory
path = os.getcwd()
parDir = path

In [15]:
# read bank entities GCIF as listt
BE = pd.read_excel(parDir+'\\Bank List\\Bank_Lists.xlsx', sheet_name='Sheet1')
BE = BE['GCIF_NO'].drop_duplicates().tolist()

In [16]:
# get bank entities funding data
MFBank = MF[MF['GCIF_NO'].isin(BE)]
MFBank = MFBank[MFBank['SEGMENT'].str.contains('GB')]

In [17]:
# filter all MF to include only active and proper accounts
MF = MF[
    ((MF['AGREE_ID'].str.startswith('ST')) &
    (~MF['STATUS'].isin(['03','04','05'])) &
    (MF['SYSTEM_TYPE'].str.startswith('8')) &
    (MF['CTRL3']!= '797')) |
    ((MF['AGREE_ID'].str.startswith('IM')) &
    (~MF['STATUS'].isin(['04','05','08'])) &
    (MF['SYSTEM_TYPE'].str.startswith('8')) &
    (MF['CTRL3']!= '797'))
]

In [18]:
# concat bank data and cleansed master funding
MFFin = pd.concat([MF,MFBank], ignore_index=True)

In [19]:
MFFin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81033 entries, 0 to 81032
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   BASE_DT             81033 non-null  int32         
 1   BASE_DT_PARSED      81033 non-null  datetime64[ns]
 2   BASE_YM             81033 non-null  object        
 3   AGREE_ID            81033 non-null  object        
 4   FLAG                81033 non-null  object        
 5   ACCT_NO             81033 non-null  object        
 6   ACCT_BR             81033 non-null  object        
 7   GCIF_CREATE_BR      18645 non-null  object        
 8   REGION              81032 non-null  object        
 9   AREA                81032 non-null  object        
 10  BRANCH              81032 non-null  object        
 11  GCIF_NO             81033 non-null  object        
 12  CIF_NO              81033 non-null  object        
 13  CUST_TYPE           18645 non-null  object    

## 2. Local Files

In [27]:
def get_last_sunday(year_month):
    # Parse the input year_month in YYYYMM format
    year = int(year_month[:4])
    month = int(year_month[4:6])

        # Get the first day of the next month
    if month == 12:  # Handle December case
        first_day_next_month = datetime.datetime(year + 1, 1, 1)
    else:
        first_day_next_month = datetime.datetime(year, month + 1, 1)

    # Get the last day of the current month
    last_day_of_month = first_day_next_month - datetime.timedelta(days=1)

    # Calculate the number of days to the previous Sunday
    days_to_sunday = last_day_of_month.weekday()

    # Get the last Sunday date
    last_sunday = last_day_of_month - datetime.timedelta(days=days_to_sunday)

    # Return the date in YYYYMMDD format
    return last_sunday.strftime('%Y%m%d')

In [21]:
RMR_Target= pd.DataFrame(columns=['BASE_DT','BASE_YM','FLAG','SEGMENT',
                                  'PROD_TYPE','BASE_AMT_FIX','SOURCE'])

In [22]:
# RMR NR Dir
RMR_NR_Dir = parDir+'\\RMR\\Non-Retail\\'
RMR_NR_Files = list(filter(os.path.isfile, glob.glob(RMR_NR_Dir + '*')))
RMR_NR_Files.sort(key=lambda x:os.path.getmtime(x))
RMR_NR_Files = [x for x in RMR_NR_Files if '~' not in x]

In [23]:
RMR_NR_Files

["d:\\Work\\Process Automation\\Daily Funding\\RMR\\Non-Retail\\RMR Oct'24 CFS Non Retail sent to PDA.xlsx",
 "d:\\Work\\Process Automation\\Daily Funding\\RMR\\Non-Retail\\RMR Nov'24 CFS Non Retail sent to PDA.xlsx",
 "d:\\Work\\Process Automation\\Daily Funding\\RMR\\Non-Retail\\RMR Dec'24 CFS Non Retail sent to PDA.xlsx"]

In [24]:
RMR_GB_Dir = parDir+'\\RMR\\GB\\'
RMR_GB_Files = list(filter(os.path.isfile, glob.glob(RMR_GB_Dir + '*')))
RMR_GB_Files.sort(key=lambda x:os.path.getmtime(x))
RMR_GB_Files = [x for x in RMR_GB_Files if '~' not in x]

In [25]:
RMR_GB_Files

["d:\\Work\\Process Automation\\Daily Funding\\RMR\\GB\\RMR Oct'24 GB sent to PDA.xlsx",
 "d:\\Work\\Process Automation\\Daily Funding\\RMR\\GB\\RMR Nov'24 GB sent to PDA.xlsx",
 "d:\\Work\\Process Automation\\Daily Funding\\RMR\\GB\\RMR Dec'24 GB sent to PDA.xlsx"]

## a. RMR Non_Retail

In [28]:
# format data into table as required
RMRSheetNames = ['RSME','BB','SME+','Micro']
Segments = ['CFS-NONRB-RSME','CFS-NONRB-BB',
            'CFS-NONRB-SME+','CFS-NONRB-MICRO']
prodTypes = ['CA','SA','TD']
Flags = ['Conven','Sharia']
Kinds = ['RMR','TARGET']
rowResult = 0
for i in range(4): # for every sheet/segment
    data = pd.read_excel(RMR_NR_Files[-1],
                     sheet_name=RMRSheetNames[i],
                     usecols='F,H',
                     skiprows=4,
                     nrows=28)
    BASE_DT = get_last_sunday(data.iloc[0,0].strftime('%Y%m%d'))
    for j in range(2): # for every kind (RMR/Target)
        col = j
        for k in range(2): # for every flag (Conven/Sharia)
            row  = 21 if k==0 else 25
            for l in range(3): # for every prod type (CA/SA/TD)
                RMR_Target.loc[rowResult,'BASE_DT'] = BASE_DT
                RMR_Target.loc[rowResult,'BASE_YM'] = BASE_DT[:6]
                RMR_Target.loc[rowResult,'FLAG'] = Flags[k]
                RMR_Target.loc[rowResult,'SEGMENT'] = Segments[i]
                RMR_Target.loc[rowResult,'PROD_TYPE'] = prodTypes[l]
                RMR_Target.loc[rowResult,'BASE_AMT_FIX'] = data.iloc[row,col]*1000000
                RMR_Target.loc[rowResult,'SOURCE'] = Kinds[j]
                row+=1
                rowResult+=1


In [29]:
RMR_Target.shape

(48, 7)

## b. RMR GB

In [30]:
# format data into table as required
rowResult = 48
data = pd.read_excel(RMR_GB_Files[-1],
                sheet_name='Corp',
                usecols='F,H',
                skiprows=4,
                nrows=28)
BASE_DT = get_last_sunday(data.iloc[0,0].strftime('%Y%m%d'))
for j in range(2): # for every kind (RMR/Target)
    col = j
    for k in range(2): # for every flag (Conven/Sharia)
        row  = 21 if k==0 else 25
        for l in range(3): # for every prod type (CA/SA/TD)
            RMR_Target.loc[rowResult,'BASE_DT'] = BASE_DT
            RMR_Target.loc[rowResult,'BASE_YM'] = BASE_DT[:6]
            RMR_Target.loc[rowResult,'FLAG'] = Flags[k]
            RMR_Target.loc[rowResult,'SEGMENT'] = 'XXX-GB-CORP'
            RMR_Target.loc[rowResult,'PROD_TYPE'] = prodTypes[l]
            RMR_Target.loc[rowResult,'BASE_AMT_FIX'] = data.iloc[row,col]*1000000
            RMR_Target.loc[rowResult,'SOURCE'] = Kinds[j]
            row+=1
            rowResult+=1

In [31]:
RMR_Target.shape

(60, 7)

In [32]:
RMR_Target['BASE_AMT_FIX'] = RMR_Target['BASE_AMT_FIX'].astype(float)

In [33]:
RMR_Target['BASE_DT_PARSED'] = pd.to_datetime(RMR_Target['BASE_DT'])

In [34]:
RMR_Target[['BASE_DT','BASE_YM','FLAG','SEGMENT','PROD_TYPE','BASE_AMT_FIX','SOURCE']]

,BASE_DT,BASE_YM,FLAG,SEGMENT,PROD_TYPE,BASE_AMT_FIX,SOURCE
0,20241230,202412,Conven,CFS-NONRB-RSME,CA,8.492773e+12,RMR
1,20241230,202412,Conven,CFS-NONRB-RSME,SA,2.840129e+11,RMR
2,20241230,202412,Conven,CFS-NONRB-RSME,TD,5.049452e+12,RMR
3,20241230,202412,Sharia,CFS-NONRB-RSME,CA,2.657584e+12,RMR
4,20241230,202412,Sharia,CFS-NONRB-RSME,SA,3.457695e+11,RMR
5,20241230,202412,Sharia,CFS-NONRB-RSME,TD,3.868954e+12,RMR
6,20241230,202412,Conven,CFS-NONRB-RSME,CA,9.918143e+12,TARGET
7,20241230,202412,Conven,CFS-NONRB-RSME,SA,2.613510e+11,TARGET
8,20241230,202412,Conven,CFS-NONRB-RSME,TD,2.905422e+12,TARGET
9,20241230,202412,Sharia,CFS-NONRB-RSME,CA,2.037567e+12,TARGET


# B. Processing

## 1. MF Aggregate

In [35]:
MF.columns

Index(['BASE_DT', 'BASE_DT_PARSED', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO',
       'ACCT_BR', 'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO',
       'CIF_NO', 'CUST_TYPE', 'PROD_NM', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME',
       'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD', 'RATE_DPK', 'BASE_AMT_FIX',
       'MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 'NPK_SALES', 'COLT',
       'BASE_AMT_ACCUM_MTD', 'INT_EXP_ACCUM_MTD', 'COF_MTD', 'DTD', 'MTD',
       'YTD', 'HIGH_COF_FLAG', 'SEGMENT_FIX', 'LOB_SORT', 'CASA_TD', 'DTD_10B',
       'MTD_10B', 'DIVISION', 'STATUS', 'SYSTEM_TYPE', 'CTRL3', 'BLOCK',
       'SOURCE'],
      dtype='object')

In [36]:
# aggregate columns
MF_Agg = MF.groupby(['BASE_DT', 'BASE_YM', 'FLAG', 'SEGMENT', 'SEGMENT_FIX', 'PROD_TYPE', 'DIVISION'], dropna=False)['BASE_AMT_FIX'].agg(BASE_AMT_FIX = 'sum')
MF_Agg.reset_index(inplace=True)

In [37]:
MF_Agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BASE_DT       39 non-null     int32  
 1   BASE_YM       39 non-null     object 
 2   FLAG          39 non-null     object 
 3   SEGMENT       39 non-null     object 
 4   SEGMENT_FIX   39 non-null     object 
 5   PROD_TYPE     39 non-null     object 
 6   DIVISION      39 non-null     object 
 7   BASE_AMT_FIX  39 non-null     float64
dtypes: float64(1), int32(1), object(6)
memory usage: 2.4+ KB


In [38]:
MF_Agg['SOURCE'] = 'TBL_BAL_SUMMARY'

In [39]:
MF_All = pd.concat([MFFin,MF_Agg], ignore_index=True)
MF_All['BASE_DT'] = MF_All['BASE_DT'].astype(int)
MF_All['BASE_YM'] = MF_All['BASE_YM'].astype(int)

In [40]:
MF_All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81072 entries, 0 to 81071
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   BASE_DT             81072 non-null  int32         
 1   BASE_DT_PARSED      81033 non-null  datetime64[ns]
 2   BASE_YM             81072 non-null  int32         
 3   AGREE_ID            81033 non-null  object        
 4   FLAG                81072 non-null  object        
 5   ACCT_NO             81033 non-null  object        
 6   ACCT_BR             81033 non-null  object        
 7   GCIF_CREATE_BR      18645 non-null  object        
 8   REGION              81032 non-null  object        
 9   AREA                81032 non-null  object        
 10  BRANCH              81032 non-null  object        
 11  GCIF_NO             81033 non-null  object        
 12  CIF_NO              81033 non-null  object        
 13  CUST_TYPE           18645 non-null  object    

## 2. RMR & Target

DB data source unavailable. Requires manual file update.

In [41]:
# create RMR container with identical columns to MF
RMRContainer = MF.iloc[0:0]

In [42]:
# Create Lob Mapping
lobMap = pd.DataFrame({'SEGMENT':["CFS-NONRB-MICRO", "CFS-NONRB-RSME", "CFS-NONRB-SME+", "CFS-NONRB-BB", "XXX-GB-CORP"],
                       'SEGMENT_FIX':["CFS-SMER & MICRO","CFS-SMER & MICRO", "CFS-SME+", "CFS-BB", "GB-CORP"]})

In [43]:
# merge LoB Mapping
RMR_Target = RMR_Target.merge(lobMap, on='SEGMENT', how='left')

In [44]:
# concat data to empty df
RMR_Target = pd.concat([RMRContainer, RMR_Target])

# C. Masking

In [45]:
# container for masked acct and agr
acct_fix = []
agr_fix = []

# mask acct and agr
for i in list(MF_All.index):
    agree = str(MF_All.loc[i,'AGREE_ID'])
    a = agree[:-7] + "XXXX" + agree[-3:]
    acct = str(MF_All.loc[i, "ACCT_NO"])
    b = acct[:-7] + "XXXX" + acct[-3:]
    agr_fix.append(a)
    acct_fix.append(b)

# replace original field with masked values
MF_All['AGREE_ID'] = agr_fix
MF_All['ACCT_NO'] = acct_fix

In [46]:
# name masking function
def masking_name(param):
    if param not in ["nan","NaN"]:
        return  ' '.join([item.replace(item[1:-1], "*"*len(item[1:-1])) if len(item)>2 and item[-1]!="," and item[-1]!="." else (item.replace(item[1:-2], "*"*len(item[1:-2])) if len(item)>2 and (item[-1]==",") else (item if len(item)>2 and (item[-1]==".") else item.replace(item[1:-2], "*"*len(item[1:-2])))) for item in param.split()])
    else:
        return param

In [47]:
# mask names
MF_All["GCIF_NAME"] = MF_All["GCIF_NAME"].astype(str)
MF_All["GCIF_NAME"] = MF_All["GCIF_NAME"].apply(masking_name) 

In [48]:
MF_All.columns

Index(['BASE_DT', 'BASE_DT_PARSED', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO',
       'ACCT_BR', 'GCIF_CREATE_BR', 'REGION', 'AREA', 'BRANCH', 'GCIF_NO',
       'CIF_NO', 'CUST_TYPE', 'PROD_NM', 'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME',
       'PROD_TYPE', 'CURR_CODE', 'SUB_LOB_CD', 'RATE_DPK', 'BASE_AMT_FIX',
       'MTD_AVG_AMT_FIX', 'MTD_INT_EXP', 'NPK_SALES', 'COLT',
       'BASE_AMT_ACCUM_MTD', 'INT_EXP_ACCUM_MTD', 'COF_MTD', 'DTD', 'MTD',
       'YTD', 'HIGH_COF_FLAG', 'SEGMENT_FIX', 'LOB_SORT', 'CASA_TD', 'DTD_10B',
       'MTD_10B', 'DIVISION', 'STATUS', 'SYSTEM_TYPE', 'CTRL3', 'BLOCK',
       'SOURCE'],
      dtype='object')

In [49]:
# Assign MF as currfin with selected columns
currFin = MF_All[['BASE_DT', 'BASE_DT_PARSED', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO',
       'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO', 'CUST_TYPE', 'PROD_NM',
       'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE',
       'COLT', 'RATE_DPK', 'BASE_AMT_FIX', 'MTD_AVG_AMT_FIX',
       'DTD', 'MTD', 'YTD', 'DIVISION', 'SOURCE', 'SEGMENT_FIX',
       'BASE_AMT_ACCUM_MTD', 'INT_EXP_ACCUM_MTD',
       'COF_MTD', 'HIGH_COF_FLAG', 'LOB_SORT', 'CASA_TD', 'DTD_10B',
       'MTD_10B','BLOCK']]

In [50]:
# Assign to RMR and Target with selected columns
RMR_Target = RMR_Target[['BASE_DT', 'BASE_DT_PARSED', 'BASE_YM', 'AGREE_ID', 'FLAG', 'ACCT_NO',
       'REGION', 'AREA', 'BRANCH', 'GCIF_NO', 'CIF_NO', 'CUST_TYPE', 'PROD_NM',
       'SUB_PROD_NM', 'SEGMENT', 'GCIF_NAME', 'PROD_TYPE', 'CURR_CODE',
       'COLT', 'RATE_DPK', 'BASE_AMT_FIX', 'MTD_AVG_AMT_FIX',
       'DTD', 'MTD', 'YTD', 'DIVISION', 'SOURCE', 'SEGMENT_FIX',
       'BASE_AMT_ACCUM_MTD', 'INT_EXP_ACCUM_MTD',
       'COF_MTD', 'HIGH_COF_FLAG', 'LOB_SORT', 'CASA_TD', 'DTD_10B',
       'MTD_10B','BLOCK']]

In [51]:
# remove possible duplicates
print(currFin.duplicated().sum())
currFin.drop_duplicates(inplace=True)

31


C:\Users\MR38804\AppData\Local\Temp\ipykernel_8252\1768179826.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currFin.drop_duplicates(inplace=True)


In [52]:
currFin.head()

,BASE_DT,BASE_DT_PARSED,BASE_YM,AGREE_ID,FLAG,ACCT_NO,REGION,AREA,BRANCH,GCIF_NO,CIF_NO,CUST_TYPE,PROD_NM,SUB_PROD_NM,SEGMENT,...,DTD,MTD,YTD,DIVISION,SOURCE,SEGMENT_FIX,BASE_AMT_ACCUM_MTD,INT_EXP_ACCUM_MTD,COF_MTD,HIGH_COF_FLAG,LOB_SORT,CASA_TD,DTD_10B,MTD_10B,BLOCK
0,20250116,2025-01-16,202501,IM16IDR703870XXXX009,Sharia,870XXXX009,JAKARTA 1,THAMRIN,KCI THAMRIN,G000214338,0012623213,INDV,Maybank Tabungan Maksi iB,Maybank Tabungan MAKSI iB,CFS-NONRB-RSME,...,0.0,0.00,0.00,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,2.530298e+08,6.325746e+05,0.002500,None,1.0,CASA,NO,NO,None
1,20250116,2025-01-16,202501,ST16IDR1670000116XXXX861,Conven,0000116XXXX861,JAWA BARAT,JAWA BARAT 4,KCI TASIKMALAYA,G000359935,0006642982,INDV,Maybank Tabungan Woman One,Maybank Tabungan Woman One,CFS-NONRB-RSME,...,0.0,-13136052.00,-13136052.00,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,4.309411e+08,1.077353e+06,0.002500,None,1.0,CASA,NO,NO,None
2,20250116,2025-01-16,202501,IM16USD167216XXXX720,Conven,216XXXX720,JAWA BARAT,JAWA BARAT 4,KCI TASIKMALAYA,G000359935,0006642982,INDV,Maybank Super Valas,Maybank Tabungan SuperValas,CFS-NONRB-RSME,...,34315.6,227340.85,227340.85,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,1.531415e+08,0.000000e+00,0.000000,None,1.0,CASA,NO,NO,None
3,20250116,2025-01-16,202501,IM16USD138213XXXX029,Conven,213XXXX029,JAKARTA 2,KOTA,KCI KOTA,G000455824,0000023773,None,Maybank Giro,Maybank Giro,CFS-NONRB-RSME,...,320438.0,2042426.75,2042426.75,UNDEFINED,TBL_BAL,CFS-SMER & MICRO,1.430031e+09,0.000000e+00,0.000000,NORMAL COF (IDR ≤1.5%&USD ≤0.25%),1.0,CASA,NO,NO,None
4,20250116,2025-01-16,202501,ST16IDR7943000007XXXX926,Sharia,3000007XXXX926,JAKARTA 3,BEKASI,KCI BEKASI,G000456624,0000044764,None,Maybank Deposito,Maybank Deposito iB,CFS-NONRB-BB,...,0.0,0.00,0.00,UNDEFINED,TBL_BAL,CFS-BB,1.762019e+09,7.208261e+07,0.040909,None,3.0,TD,NO,NO,1


In [53]:
# Data date
MF_date = currFin.loc[0,'BASE_DT']
RMR_Target_Date = RMR_Target.loc[0,'BASE_YM']

In [54]:
currFin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81041 entries, 0 to 81071
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   BASE_DT             81041 non-null  int32         
 1   BASE_DT_PARSED      81002 non-null  datetime64[ns]
 2   BASE_YM             81041 non-null  int32         
 3   AGREE_ID            81041 non-null  object        
 4   FLAG                81041 non-null  object        
 5   ACCT_NO             81041 non-null  object        
 6   REGION              81001 non-null  object        
 7   AREA                81001 non-null  object        
 8   BRANCH              81001 non-null  object        
 9   GCIF_NO             81002 non-null  object        
 10  CIF_NO              81002 non-null  object        
 11  CUST_TYPE           18639 non-null  object        
 12  PROD_NM             81002 non-null  object        
 13  SUB_PROD_NM         81002 non-null  object        


# D. RMR & Target Append

In [55]:
# Load YMKeeper
YMRecordPrev = pd.read_csv(parDir+'/Keeper/YMKeeper.csv')
RMRTargetRecordYM = YMRecordPrev.loc[0,'RMRTarget_YM']

In [56]:
# concat RMR Target if current YM > saved YM
if int(RMR_Target.loc[0,'BASE_YM']) > RMRTargetRecordYM:
    currFin = pd.concat([currFin,RMR_Target],ignore_index=True)
else:
    pass

In [57]:
currFin['SOURCE'].value_counts()

SOURCE
TBL_BAL            81002
TBL_BAL_SUMMARY       39
Name: count, dtype: int64

In [58]:
# save data
currFin.to_csv(parDir+f'\\Temp_Result\\MF_{MF_date}.csv', index=False)

# E. Save Result

## 1. Connect to SFTP

In [59]:
# SFTP connection details
hostname = '10.220.42.38'
port = '22'
username = os.environ['UID']
password = os.environ['UPW']

## 2. Save to SFTP

In [60]:
# create dir if not exist
def create_remote_directory(sftp, remote_path):
    """
    Recursively create directories on the SFTP server if they do not exist.
    :param sftp: SFTP session object.
    :param remote_path: Full path of the directory to create.
    """
    dirs = remote_path.split('/')
    current_path = ""
    for dir_name in dirs:
        if dir_name:  # Skip empty parts
            current_path += f"/{dir_name}"
            try:
                sftp.stat(current_path)  # Check if the directory exists
            except FileNotFoundError:
                sftp.mkdir(current_path)  # Create the directory
                print(f"Directory created: {current_path}")

In [61]:
# sftp hostname and credential
hostnameSFTP = '10.220.42.38'
portSFTP = '22'
usernameSFTP = os.environ['UID']
passwordSFTP = os.environ['UPW']

In [62]:
# sava dataframe as csv to Memory
csv_buffer = io.StringIO()
currFin.to_csv(csv_buffer, index=False)
csv_buffer.seek(0) # reset buffer position to beginning

# Save path
remote_path = f'/PDA/PNR Automation/Daily Funding/MF_{MF_date}.csv'
remote_directory = '/PDA'

# Estabilish SFTP connection
try:
    # Create the SSH client
    ssh = paramiko.SSHClient()
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    ssh.connect(hostnameSFTP, port=portSFTP, username=usernameSFTP, password=passwordSFTP)

    # Open SFTP session
    sftp = ssh.open_sftp()

    # Ensure the directory exists
    create_remote_directory(sftp, remote_directory)

    # Write the buffer to the remote file
    with sftp.file(remote_path, 'w') as remote_file:
        remote_file.write(csv_buffer.getvalue())

    logging.info(f'File uploaded successfully to {remote_path}')
except Exception as e:
    logging.error("Exception occurred", exc_info=True)
finally:
    if 'sftp' in locals():
        sftp.close()
    if 'ssh' in locals():
        ssh.close()

2025-01-17 15:08:08 - INFO - Connected (version 2.0, client 6.6.2.451)
2025-01-17 15:08:10 - INFO - Authentication (password) successful!
2025-01-17 15:08:10 - INFO - [chan 0] Opened sftp connection (server version 3)
2025-01-17 15:09:10 - INFO - File uploaded successfully to /PDA/PNR Automation/Daily Funding/MF_20250116.csv
2025-01-17 15:09:10 - INFO - [chan 0] sftp session closed.


## 3. Save YM Keeper

In [63]:
# save RMR and Target YM
YMRecord = pd.DataFrame({'RMRTarget_YM':[RMR_Target.loc[0,'BASE_YM']]})
YMRecord.to_csv(parDir+'/Keeper/YMKeeper.csv', index=False)

In [64]:
YMRecord

,RMRTarget_YM
0,202412


In [65]:
logging.info('Processing Finished')

2025-01-17 15:09:10 - INFO - Processing Finished
